### Runnable
* Runnable은 런타임에 실행될 수 있는 모든 객체를 의미한다.

### Runnable 종류
* invoke : 입력에 대해 체인을 호출한다.
* stream : 응답의 청크를 스트리밍한다.
* batch : 입력 목록에 대한 일괄 처리를 수행한다.

이외에 비동기 메소드도 존재한다.
* astream : 비동기적으로 응답의 청크를 비동기 스트리밍한다.
* ainvoke : 비동기적으로 입력에 대해 체인을 호출한다.
* abatch : 비동기적으로 입력 목록에 대해 일괄 처리를 수행한다.
* astream_log : 최종 응답 및 발생하는 중간단계를 스트리밍한다.

In [1]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="../.env")

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.1
)

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{lecture}에 대해 3문장으로 설명해줘")

chain = prompt | llm | StrOutputParser()

### invoke

In [4]:
print(chain.invoke({"lecture" : "리엑트"}))

리액트(React)는 사용자 인터페이스를 구축하기 위한 JavaScript 라이브러리로, 페이스북에서 개발하였습니다. 컴포넌트 기반 아키텍처를 사용하여 재사용 가능한 UI 구성 요소를 만들 수 있으며, 가상 DOM을 통해 효율적인 렌더링을 지원합니다. 리액트는 단일 페이지 애플리케이션(SPA) 개발에 특히 유용하며, 다양한 생태계와 도구를 통해 확장성과 유연성을 제공합니다.


### stream [실시간출력]

* 데이터 스트림을 생성하고, 스트림을 반복하여 각 데이터의 내용을 즉시 출력한다.
* end="" 인자를 사용해 줄바꿈을 하지 않을 수 있다.
* flush=True 인자를 사용해 즉시 출력 가능

In [6]:
for token in chain.stream({"lecture" : " Python"}):
    
    print(token, end="", flush=True)

Python은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 분야에서 널리 사용됩니다. 객체 지향, 절차적, 함수형 프로그래밍 패러다임을 지원하며, 방대한 라이브러리와 프레임워크를 통해 데이터 분석, 웹 개발, 인공지능 등 다양한 응용 프로그램을 쉽게 개발할 수 있습니다. 또한, 플랫폼 독립적이어서 Windows, macOS, Linux 등 여러 운영 체제에서 실행할 수 있습니다.

### batch [단위 실행]
* 여러개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각각의 입력에 대한 체임을 실행한다.
* max_concurrency : 최대 병렬 처리 수를 지정할 수 있다.

In [14]:
result = chain.batch(
    [
        {"lecture": "파이썬"},
        {"lecture": "자바"},
        {"lecture": "자바스크립트"},
        {"lecture": "SQL"}
    ],
    config={"max_concurrency": 3}
)

print(result)

['파이썬은 간결하고 읽기 쉬운 문법을 가진 고급 프로그래밍 언어로, 다양한 분야에서 널리 사용됩니다. 데이터 분석, 웹 개발, 인공지능 등 여러 응용 프로그램을 지원하는 방대한 라이브러리와 프레임워크를 제공합니다. 또한, 플랫폼 독립적이어서 다양한 운영 체제에서 실행할 수 있는 장점이 있습니다.', '자바는 객체 지향 프로그래밍 언어로, 플랫폼 독립성을 제공하여 "한 번 작성하면 어디서나 실행"할 수 있는 특징이 있습니다. 강력한 메모리 관리와 자동 가비지 수집 기능을 통해 개발자가 메모리 관리를 신경 쓰지 않고도 안정적인 애플리케이션을 개발할 수 있게 합니다. 또한, 풍부한 라이브러리와 프레임워크를 통해 다양한 분야의 소프트웨어 개발에 널리 사용되고 있습니다.', '자바스크립트는 웹 페이지에 동적인 기능을 추가하기 위해 사용되는 프로그래밍 언어입니다. 클라이언트 측에서 실행되며, HTML과 CSS와 함께 웹 개발의 핵심 기술 중 하나로 자리잡고 있습니다. 또한, Node.js와 같은 런타임 환경을 통해 서버 측 프로그래밍에도 활용될 수 있습니다.', 'SQL(Structured Query Language)은 데이터베이스 관리 시스템에서 데이터를 정의, 조작, 검색하기 위해 사용되는 표준 프로그래밍 언어입니다. SQL을 사용하면 데이터베이스에 저장된 정보를 쉽게 조회하고, 삽입, 업데이트, 삭제할 수 있습니다. 또한, 데이터베이스의 구조를 정의하고, 사용자 권한을 관리하는 등의 작업도 수행할 수 있습니다.']


---

### Parallel (병렬성)
* LCEL를 사용하여 체인을 구성할때, 여러 체인을 동식에 실행 할 수 있다.

In [15]:
chain1 = (
    PromptTemplate.from_template("{country} 의 수도는 어디야?")
    | llm
    | StrOutputParser()
)

chain2 = (
    PromptTemplate.from_template("{country} 의 면적은 얼마야?")
    | llm
    | StrOutputParser()
)

In [16]:
from langchain_core.runnables import RunnableParallel

# 병렬 실행 체인
combined = RunnableParallel(capital=chain1, area=chain2)

In [17]:
print(chain1.invoke({"country" :"대한민국"}))

대한민국의 수도는 서울입니다.


In [18]:
print(chain2.invoke({"country" :"미국"}))

미국의 면적은 약 9,830,000 평방킬로미터(약 3,796,000 평방마일)입니다. 이는 세계에서 세 번째로 큰 나라로, 러시아와 캐나다에 이어 가장 큰 면적을 가지고 있습니다.


In [20]:
result = combined.invoke({"country" : "대한민국"})

print(result)

{'capital': '대한민국의 수도는 서울입니다.', 'area': '대한민국의 면적은 약 100,210 평방킬로미터(㎢)입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과 함께 한반도를 구성하고 있습니다.'}


In [21]:
print(result["capital"])
print(result["area"])

대한민국의 수도는 서울입니다.
대한민국의 면적은 약 100,210 평방킬로미터(㎢)입니다. 이는 한반도의 남쪽 부분에 해당하며, 북한과 함께 한반도를 구성하고 있습니다.


### RunnablePassThrough
* 입력을 그대로 전달하는 역할
* 단독으로 호출될 경우 단순히 입력을 받아 그대로 전달

In [22]:
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0.0
)

prompt = PromptTemplate.from_template("{num}의 약수를 알려줘")

chain = prompt | llm | StrOutputParser()

In [23]:
# invoke()를 통해 실행할때는 입력이 딕셔너리여야 하지만,
# 1개의 변수만 템플릿에 작성이 되었다며, 값만 전달해도 된다.
chain.invoke(10)

'10의 약수는 1, 2, 5, 10입니다.'

In [25]:
from langchain_core.runnables import RunnablePassthrough

passthrough_chain = {"num": RunnablePassthrough()} | prompt | llm | StrOutputParser()

passthrough_chain.invoke({"num" : 10})

'10의 약수는 1, 2, 5, 10입니다.'

**RunnablePassthrough,assign()**
* 입력값으로 들어온 값의 key/value 쌍을 새롭게 할당된 key/value 쌍을 합쳐준다.

In [26]:
RunnablePassthrough.assign(new_date=lambda x: x["num"] * 3).invoke({"num" : 1})

{'num': 1, 'new_date': 3}

### RunnableLabda
* 입력 값에 대한 추가 처리를 수행할 수 있다.
* 람다 함수를 사용해 입력 값에 대한 추가 처리를 진행
* 날씨, 시간

In [31]:
from datetime import datetime

def get_today(num):
    print("num :", num)
    return datetime.now().strftime("%b-%d")

# get_today()

In [32]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# prompt
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 알려줘"
)

# llm
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# chain 생성

chain = (
    {"today" : RunnableLambda(get_today), "n" : itemgetter("n")}
    | prompt
    |llm
    |StrOutputParser()
)

In [36]:
print(chain.invoke({"n" : 3}))

num : {'n': 3}
10월 28일이 생일인 유명인 중 몇 명을 소개하자면:

1. 빌 게이츠 (Bill Gates) - 마이크로소프트의 공동 창립자이자 세계적인 기업가입니다.
2. 줄리아 로버츠 (Julia Roberts) - 아카데미상을 수상한 미국의 유명한 배우입니다.
3. 호아킨 피닉스 (Joaquin Phoenix) - 여러 영화에서 뛰어난 연기를 선보인 미국의 배우입니다.

이 외에도 많은 유명인들이 있지만, 이 세 명이 대표적입니다.
